In [1]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [4]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import re
from transformers import TextStreamer
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
model_name = "cychiuak/llama3_combined_V1"
validation_data = pd.read_csv('combined_test.csv', sep='\t')
print(validation_data)

                                                   text     label
0     $HCP Come to the party and buy this -gonna giv...  positive
1     @gakrum nice chart shows distinctive down chan...  negative
2     Japan's Asahi to submit bid next week for SABM...  positive
3     Tesla Motors recalls 2,700 Model X SUVs $TSLA ...  negative
4          CRH's concrete bid for Holcim Lafarge assets  positive
...                                                 ...       ...
6580  Stocks making the biggest moves midday: TD Ame...   neutral
6581  Stocks making the biggest moves premarket: Fit...   neutral
6582  Stocks making the biggest moves premarket: Hom...   neutral
6583  Stocks making the biggest moves premarket: TD ...   neutral
6584         TCO, NNVC, GPOR and JE among midday movers   neutral

[6585 rows x 2 columns]


In [6]:
alpaca_prompt = """Below is a classification task. Answer only positive, neutral or negative, do not say anything else .

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
        "Although this stock recorded a history high revenue, I don't think it will last long.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<|begin_of_text|>Below is a classification task. Answer only positive, neutral or negative, do not say anything else.

### Instruction:
### Classify the sentiment of the following sentence as negative, neutral or positive.

### Input:
Although this stock recorded a history high revenue, I don't think it will last long.

### Response:
mildly negative<|end_of_text|>


In [9]:
answers = []
for i in range(len(validation_data['text'])):
  print("i is ",i)
# for i in range(5):
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
          validation_data['text'][i], # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  textOutput = tokenizer.batch_decode(outputs)
  print("textOutput is",textOutput)
  match = re.search(r"### Response:\n(.*?)<\|end_of_text\|>", textOutput[0], re.DOTALL)

  if match:
      response_text = match.group(1).strip()
      print("response_text is ",response_text)
      response_text = response_text.lower()
      if("positive" in response_text):
        print("positive")
        answers.append("positive")
      elif("neutral" in response_text):
        print("neutral")
        answers.append("neutral")
      elif("negative" in response_text):
        print("negative")
        answers.append("negative")
      else:
        answers.append("error")
        print("Response section not found.")
      print("label is",validation_data['label'][i])


Streaming output truncated to the last 5000 lines.
i is  5585
textOutput is ['<|begin_of_text|>Below is a classification task. Answer only positive, neutral or negative, do not say anything else.\n\n### Instruction:\n### Classify the sentiment of the following sentence as negative, neutral or positive.\n\n### Input:\nWhy Accelerated Approval Could Be Huge for This Ovarian Cancer Study\n\n### Response:\npositive<|end_of_text|>']
response_text is  positive
positive
label is neutral
i is  5586
textOutput is ['<|begin_of_text|>Below is a classification task. Answer only positive, neutral or negative, do not say anything else.\n\n### Instruction:\n### Classify the sentiment of the following sentence as negative, neutral or positive.\n\n### Input:\nwhy macro funds are shutting down left and right https://t.co/gVYowOBmqe\n\n### Response:\nneutral<|end_of_text|>']
response_text is  neutral
neutral
label is neutral
i is  5587
textOutput is ['<|begin_of_text|>Below is a classification task. Answ

In [11]:

correctPre = 0
for i in range(len(answers)):
  ground_truth = validation_data['label'][i]
  # if ground_truth == "moderately positive":
  #     print("detected positive")
  #     ground_truth = "positive"
  # if ground_truth == "mildly positive":
  #     print("Detected negative")
  #     ground_truth = "positive"
  # if ground_truth == "moderately negative":
  #     print("detected positive")
  #     ground_truth = "negative"
  # if ground_truth == "mildly negative":
  #     print("Detected negative")
  #     ground_truth = "negative"
  if "negative" in ground_truth:
      ground_truth = "negative"
  if "positive" in ground_truth:
      ground_truth = "positive"
  print("ground_truth is",ground_truth)
  print("prediction is ",answers[i])
  if(answers[i] == ground_truth):
    print("correct")
    correctPre += 1
print(correctPre)
accuracy = correctPre / len(answers)
print("accuracy is",accuracy)

Streaming output truncated to the last 5000 lines.
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth i

In [ ]:
userInput = "The stock price is about to go down!"
inputs = tokenizer(
[
    alpaca_prompt.format(
        "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
        userInput, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
textOutput = tokenizer.batch_decode(outputs)
output = re.search(r"### Response:\n(.*?)<\|end_of_text\|>", textOutput[0], re.DOTALL).group(1).strip()
print("input is ",userInput)
print("output is ",output)

In [12]:
for i in range(len(validation_data)):
  # if ground_truth == "moderately positive":
  #     print("detected positive")
  #     ground_truth = "positive"
  # if ground_truth == "mildly positive":
  #     print("Detected negative")
  #     ground_truth = "positive"
  # if ground_truth == "moderately negative":
  #     print("detected positive")
  #     ground_truth = "negative"
  # if ground_truth == "mildly negative":
  #     print("Detected negative")
  #     ground_truth = "negative"
  if "negative" in ground_truth:
      validation_data['label'][i] = "negative"
  if "positive" in ground_truth:
      validation_data['label'][i] = "positive"
  print("ground_truth is",validation_data['label'][i])
  print("prediction is ",answers[i])
  if(answers[i] == validation_data['label'][i]):
    print("correct")
    correctPre += 1

Streaming output truncated to the last 5000 lines.
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is  neutral
correct
ground_truth is neutral
prediction is 

In [13]:
from sklearn.metrics import confusion_matrix
label = validation_data['label']
print(confusion_matrix(label,answers,labels=["positive", "neutral", "negative"]))

[[ 458   62    6]
 [ 278 1725  103]
 [   6   46  345]]


In [15]:
import numpy as np

def calculate_f1_scores(confusion_matrix):
    num_classes = confusion_matrix.shape[0]
    f1_scores = []

    for i in range(num_classes):
        true_positive = confusion_matrix[i, i]
        false_positive = sum(confusion_matrix[:, i]) - true_positive
        false_negative = sum(confusion_matrix[i, :]) - true_positive

        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

    macro_f1 = np.mean(f1_scores)
    return f1_scores, macro_f1

# Example usage:
# confusion_matrix = np.array([
#     [75, 33, 0],
#     [9, 203, 3],
#     [0, 6, 49]
# ])
matrix = confusion_matrix(label,answers,labels=["positive", "neutral", "negative"])
f1_scores, macro_f1 = calculate_f1_scores(matrix)
print("F1 Scores for each class:", f1_scores)
print("Macro F1 Score:", macro_f1)

F1 Scores for each class: [0.722397476340694, 0.8758568164508759, 0.8108108108108109]
Macro F1 Score: 0.8030217012007936
